In [42]:
import imaplib
imap_ssl_host = 'imap.gmail.com'  # imap.mail.yahoo.com
imap_ssl_port = 993

mail = imaplib.IMAP4_SSL(imap_ssl_host, imap_ssl_port)
mail.login('peterdemin@gmail.com', 'Pr1m@rym@1lb0x')
mail.list()
# Out: list of "folders" aka labels in gmail.
mail.select("inbox") # connect to inbox

('OK', ['8'])

In [19]:
result, data = mail.search(None, "ALL")

In [20]:
data

['1 2 3 4 5 6 7 8 9 10']

In [21]:
ids = data[0] # data is a list.
id_list = ids.split() # ids is a space separated string
latest_email_id = id_list[-1] # get the latest
 
result, body_data = mail.fetch(latest_email_id, "(RFC822)") # fetch the email body (RFC822) for the given ID

raw_email = body_data[0][1] # here's the body, which is raw text of the whole email
# including headers and alternate payloads

In [45]:
result, inbox_data = mail.uid('search', None, "ALL") # search and return uids instead
email_uids = inbox_data[0].split()
latest_email_uid = email_uids[-1]
result, body_data = mail.uid('fetch', latest_email_uid, '(RFC822)')
raw_email = data[0][1]

In [46]:
email_uids

['30773', '33718', '33738', '33740', '33750', '33757', '33759', '33760']

In [48]:
import email.Header

In [82]:
for uid in email_uids:
    result, body = mail.uid('fetch', uid, '(RFC822)')
    raw_email = body[0][1]
    msg = email.message_from_string(raw_email)
    print('%-8s: %s' % ('UID', uid))
    for header in [ 'subject', 'to', 'from' ]:
        #print(msg[header])
        #print(email.Header.decode_header(msg[header]))
        text = email.Header.decode_header(msg[header].replace('\r\n', ' '))[0][0]
        print('%-8s: %s' % (header.upper(), text))
    print('')

UID     : 30773
SUBJECT : Check out the new Developer Hub
TO      : "peterdemin@gmail.com" <peterdemin@gmail.com>
FROM    : Open Banking <openbanking@citi.com>

UID     : 33718
SUBJECT : PETR, it's a good time to refinance your auto loan
TO      : <peterdemin@gmail.com>
FROM    : "Bank of America" <bankofamerica@emcom.bankofamerica.com>

UID     : 33738
SUBJECT : Challenge 3 💡 
TO      : <peterdemin+toothpaste.coffee@gmail.com>
FROM    : toothpaste.coffee

UID     : 33740
SUBJECT : Igor Sobolev sent you an Amazon.com Gift Card!
TO      : peterdemin@gmail.com
FROM    : "Amazon.com Gift Card" <gc-orders@gc.email.amazon.com>

UID     : 33750
SUBJECT : Challenge 4💡
TO      : <peterdemin+toothpaste.coffee@gmail.com>
FROM    : toothpaste.coffee

UID     : 33757
SUBJECT : 📱 Хиты из мира техники!
TO      : poslano@gmail.com
FROM    : Технопарк.ру - интернет-магазин

UID     : 33759
SUBJECT : A new transaction was charged to your account
TO      : peterdemin@gmail.com
FROM    : "Capital One" <c

In [84]:
result, body = mail.uid('fetch', '33760', '(RFC822)')
msg = email.message_from_string(body[0][1])
print(get_first_text_block(msg))


Hi, PETR, here's your available balance

Balance: $676.32
Account: Checking -8420
Date: April 15, 2018

Sign in to Online Banking or the mobile app to view your account
activity.

This balance may be different from what displays in Online Banking
if you've had subsequent activity on your account since we sent
this message.

Lost it? Lock it?
Lock your debit card to prevent unauthorized activity.  

Download the app now:  http://promotions.bankofamerica.com/mobile/emailcampaignM?cm_mmc=OLB-Mobile Banking-_-Mobile Banking-_-OM16LT0034_AMC_AVLBAL_0816_Get_the_App-_-AlertMC_Aug_2016

Your last sign-in was 04/15/2018
To verify that this email is from Bank of America, confirm your last 
sign-in date is correct. To access Online or Mobile Banking, go 
directly to bankofamerica.com or use our Mobile Banking app.
Remember: We never ask for private information such as an account number,
card PIN, or Social Security or Tax ID number in email messages. If you 
think an email is suspicious, don't 

```yaml
checks:
  - email: peterdemin@gmail.com
    match:
      - from: "Bank of America" <onlinebanking@ealerts.bankofamerica.com>
      - subject: Your Available Balance
    transform:
      - bash: "awk /^Balance:/ { print $2; }"
    notify:
      - stash:
          bofa: {{ content }}
      - telegram
```

In [35]:
import email
 
print email_message['To']
 
print email.utils.parseaddr(email_message['From']) # for parsing "Yuji Tomita" <yuji@grovemade.com>
 
print email_message.items() # print all headers
 
# note that if you want to get text content (body) and the email contains
# multiple payloads (plaintext/ html), you must parse each message separately.
# use something like the following: (taken from a stackoverflow post)
def get_first_text_block(email_message_instance):
    maintype = email_message_instance.get_content_maintype()
    if maintype == 'multipart':
        for part in email_message_instance.get_payload():
            if part.get_content_maintype() == 'text':
                return part.get_payload()
    elif maintype == 'text':
        return email_message_instance.get_payload()

prislano@gmail.com
('Google', 'no-reply@accounts.google.com')
[('Delivered-To', 'peterdemin@gmail.com'), ('Received', 'by 10.12.197.144 with SMTP id a16csp2775688qvj;\r\n        Sun, 15 Apr 2018 09:04:42 -0700 (PDT)'), ('X-Received', 'by 10.200.12.13 with SMTP id k13mr12734461qti.281.1523808282573;\r\n        Sun, 15 Apr 2018 09:04:42 -0700 (PDT)'), ('ARC-Seal', 'i=2; a=rsa-sha256; t=1523808282; cv=pass;\r\n        d=google.com; s=arc-20160816;\r\n        b=NrG3IsVEASwuh4LQMMuvRiTYt5Oj5lApPgG22X7baurTj80lMJof9zV/etkxV6o4cu\r\n         BMz8o+bRHSjuunxPBJlqkQUOU9PrABw5USJENQ8yiBs+VQ5o3iY7Hpo5HG4VFg/ylB/s\r\n         bXYsE6O07fo9ItM4P02e16+fYjvDYxdFk2ovzWppc4oIAH+Dnj+AkUWg5Ye2ZWjpgboc\r\n         uY5homLu2UZq6WID+Nhp/2TNQhsgOuTJYnbNBmlPRmAIAkTmIm8csC6Ms0gmJ0nSeeY0\r\n         GqZZXPFZMM3SPG3maWy0h1iFRm3Uq+MCpWDeaslbCR9UndMruFJS8yb5DWDl3pN8phAx\r\n         kvHQ=='), ('ARC-Message-Signature', 'i=2; a=rsa-sha256; c=relaxed/relaxed; d=google.com; s=arc-20160816;\r\n        h=to:from:subject:m

In [39]:
import base64
print(base64.b64decode(get_first_text_block(email_message)))

Access for less secure apps has been turned on



You received this message because prislano@gmail.com is listed as the
recovery email for peterdemin@gmail.com. If peterdemin@gmail.com is not
your Google Account, click here to disconnect
<https://accounts.google.com/AccountDisavow?adt=AOX8kioImqXluxpIRQkVFwB4BtsxNDEjkyidsa6V8fl49DgLzGSAolxAfrN8Gq0&rfn=28>
from that account and stop receiving emails.



Hi Peter,
You recently changed your security settings so that your Google Account
peterdemin@gmail.com is no longer protected by modern security standards.

Please be aware that it is now easier for an attacker to break into your
account. You can make your account safer again by undoing this change here
<https://myaccount.google.com/lesssecureapps>, then switching to apps made
by Google such as Gmail to access your account.

*Don't recognize this activity?*
Review your recently used devices
<https://accounts.google.com/AccountChooser?Email=peterdemin@gmail.com&continue=https://myaccount.